In [1]:
# Imports
import numpy as np
import pandas as pd

# pip3 install pandas ipython jupyter


In [2]:
# Alumnos

students = pd.read_csv('input/students.csv', sep=',', index_col=0, lineterminator='\n')
students.head()

# students.loc['alumno1', 'attr2']

,attr1,attr2,attr3,attr4,attr5,attr6,attr7,attr8,attr9,attr10
alumnos,,,,,,,,,,
alumno1,0.000000,0.071429,0.133333,0.1875,0.235294,0.277778,0.315789,0.35,0.380952,0.409091
alumno2,0.076923,0.142857,0.200000,0.2500,0.294118,0.333333,0.368421,0.40,0.428571,0.454545
alumno3,0.153846,0.214286,0.266667,0.3125,0.352941,0.388889,0.421053,0.45,0.476190,0.500000
alumno4,0.230769,0.285714,0.333333,0.3750,0.411765,0.444444,0.473684,0.50,0.523810,0.545455
alumno5,0.307692,0.357143,0.400000,0.4375,0.470588,0.500000,0.526316,0.55,0.571429,0.590909


In [3]:
# modificar el CSV para que lo números sean [0-1]

# students = students/students.max()
# students.head()
# students.to_csv('input/students.csv', index=True)


In [4]:
# empresas

companies = pd.read_csv('input/companies.csv', sep=',', index_col=0, lineterminator='\n')
companies.head()

,attr1,attr2,attr3,attr4,attr5,attr6,attr7,attr8,attr9,attr10
empresas,,,,,,,,,,
empresa1,0.000000,0.071429,0.133333,0.187500,0.235294,0.277778,0.315789,0.3500,0.380952,0.409091
empresa2,0.111111,0.200000,0.272727,0.333333,0.384615,0.428571,0.466667,0.5000,0.529412,0.555556
empresa3,0.222222,0.300000,0.363636,0.416667,0.461538,0.500000,0.533333,0.5625,0.588235,0.611111
empresa4,0.333333,0.400000,0.454545,0.500000,0.538462,0.571429,0.600000,0.6250,0.647059,0.666667
empresa5,0.444444,0.500000,0.545455,0.583333,0.615385,0.642857,0.666667,0.6875,0.705882,0.722222


In [5]:
# modificar el CSV para que lo números sean [0-1]

# companies = companies/companies.max()
# companies.head()
# companies.to_csv('input/companies.csv', index=True)

In [6]:
# Matching

# PESOS = (alumnos.columns)
PESOS = {'attr1' : 2, 
         'attr2' : 1, 
         'attr3' : 1, 
         'attr4' : 1, 
         'attr5' : 3, 
         'attr6' : 1, 
         'attr7' : 1, 
         'attr8' : 1,
         'attr9' : 1, 
         'attr10' : 3
        }

def calc_dist(attr_s, attr_c, p): 
    return p*(1 - abs(attr_s-attr_c))**2

def match(student, company, weights=PESOS): 
    res = sum([calc_dist(x, y, p) for x, y, p in zip(student, company, weights.values())])
    return res

In [7]:
match(students.loc['alumno1'], companies.loc['empresa1'])

15.0

# Emparejamiento

En cada ronda se crea una pareja alumno-empresa empezando por el mejor %. Luego se pasa al siguente alumno. Si la empresa que hacía mejor match es la del caso anterior, pasa a su segundo mejor match, etc. 

En cada ronda se cambia el estudiante que elige primero, entre los que no han ido primeros

In [8]:
# !jupyter kernelspec list
# import sys
# sys.path

In [9]:
# https://es.wikipedia.org/wiki/Sistema_de_todos_contra_todos




In [10]:
# create dataframe de tamañ0 SxC

matching = pd.DataFrame(np.zeros((len(students), len(companies))) , columns=companies.index, index=students.index)
matching.head()

empresas,empresa1,empresa2,empresa3,empresa4,empresa5,empresa6,empresa7,empresa8,empresa9,empresa10
alumnos,,,,,,,,,,
alumno1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alumno2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alumno3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alumno4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
alumno5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
matching.index, matching.columns

(Index(['alumno1', 'alumno2', 'alumno3', 'alumno4', 'alumno5', 'alumno6',
        'alumno7', 'alumno8', 'alumno9', 'alumno10', 'alumno11', 'alumno12',
        'alumno13', 'alumno14'],
       dtype='object', name='alumnos'),
 Index(['empresa1', 'empresa2', 'empresa3', 'empresa4', 'empresa5', 'empresa6',
        'empresa7', 'empresa8', 'empresa9', 'empresa10'],
       dtype='object', name='empresas'))

In [12]:
# matching.loc['alumno1', 'empresa3']

for a in matching.index: 
    for c in matching.columns:
        # print(a, c)
        matching.loc[a, c] = match(students.loc[a], companies.loc[c])
matching.head()

empresas,empresa1,empresa2,empresa3,empresa4,empresa5,empresa6,empresa7,empresa8,empresa9,empresa10
alumnos,,,,,,,,,,
alumno1,15.000000,11.056191,9.163737,7.458931,5.941773,4.612263,3.470400,2.516186,1.749619,1.170700
alumno2,13.306327,12.611052,10.578480,8.733555,7.076278,5.606649,4.324668,3.230334,2.323649,1.604611
alumno3,11.717633,13.871870,12.098202,10.113159,8.315763,6.706015,5.283915,4.049463,3.002659,2.143503
alumno4,10.233920,13.909148,13.654529,11.597743,9.660228,7.910362,6.348143,4.973572,3.786649,2.787374
alumno5,8.855187,12.430244,14.202871,13.187307,11.109674,9.219688,7.517351,6.002661,4.675620,3.536226


In [13]:
matching = matching/matching.max().max() # para normalizar
matching.head()

empresas,empresa1,empresa2,empresa3,empresa4,empresa5,empresa6,empresa7,empresa8,empresa9,empresa10
alumnos,,,,,,,,,,
alumno1,1.000000,0.737079,0.610916,0.497262,0.396118,0.307484,0.231360,0.167746,0.116641,0.078047
alumno2,0.887088,0.840737,0.705232,0.582237,0.471752,0.373777,0.288311,0.215356,0.154910,0.106974
alumno3,0.781176,0.924791,0.806547,0.674211,0.554384,0.447068,0.352261,0.269964,0.200177,0.142900
alumno4,0.682261,0.927277,0.910302,0.773183,0.644015,0.527357,0.423210,0.331571,0.252443,0.185825
alumno5,0.590346,0.828683,0.946858,0.879154,0.740645,0.614646,0.501157,0.400177,0.311708,0.235748


In [17]:
matching.max()

1.0

In [15]:
# matching.loc['alumno1', 'empresa3'] = 444
# matching

list(students.loc['alumno1'])

[0.0,
 0.07142857142857142,
 0.13333333333333333,
 0.1875,
 0.2352941176470588,
 0.2777777777777778,
 0.3157894736842105,
 0.35,
 0.3809523809523809,
 0.4090909090909091]

In [16]:
abs(-3)

3